# Creating a Bestiary and Adding Creatures to the Map

This is the process that loads the monster data from S3 and adds it to the mapData global data. 

Existing features:
* creature probability and danger level determines the likelyhood that an area will contain monsters
* each monster `[m]` is given specific stats, however global information for `monsters` is kept in `monsters[meta]` <br>
TODO:
* business logic (js) that will facilitate character attacks

In [1]:
import pandas as pd
import numpy as np
import yaml

### Transform and Load
First I need to transform the written bestiary from the .csv that I'm using to create creatures. 
The creatures are created in a spreadsheet and then rendered randomly on a map based on the attributes in that terrain.

Adding more features to the monsters is easy. You just need to add more columns to the file and a value for each. 

In [2]:
bdf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Bestiary.tsv",sep="\t",index_col=0)
ddf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Terrain Detail.tsv",sep="\t",index_col=0)

In [3]:
bdf

,name,health,healthMaxVariance,move,size,move type,render type,group min,group max,color,perception,attackType,damage
index,,,,,,,,,,,,,
goblin,Goblin,3,2,5,5,swarm,pack,5,10,#3E8E3E,10,swung a rusty sword,4
tiger,Tiger,8,4,10,6,swarm,alone,1,1,#e08d3c,200,swung it's might claws,12
scorpion,Giant Scorpion,8,7,3,8,swarm,scatter,1,2,#716D6F,10,struck with it's stinger,12
yeti,Yeti,15,3,5,15,swarm,alone,1,1,#a0d6b4,50,swung with it's fists,8
raider,Raider,15,5,6,10,swarm,pack,2,3,#4a3b29,150,swung a daggar,6


In [4]:
ddf

,Type Name,Type,Danger Level,Creature Probability,Treasure Level,Description,Detailed Description,creatures,Terrain Textures
Code,,,,,,,,,
fo,forest,normal,4,0.5,3,a quiet deciduous forest,NaN,goblin,"[ {'name':'Tree', 'density':6,..."
sw,swamp,normal,8,0.8,2,a desolate swamp,NaN,goblin,"[ {'name':'Fern', 'abundance':..."
mt,mountain,normal,6,0.4,5,a rugged mountain range,NaN,goblin,"[ {'name':'Boulder', 'density'..."
sa,savanna,normal,3,0.1,3,a dry savana,NaN,tiger,"[ {'name':'Umbrella Thorn', 'd..."
pl,plain,normal,2,0.1,2,a grassy plain,NaN,tiger,"[ {'name':'Grass', 'abundance'..."
ds,desert,normal,1,0.1,1,a barren desert,NaN,scorpion,"[ {'name':'Sand', 'abundance':..."
ic,tundra,normal,0,0.1,0,a tundra,NaN,yeti,"[ {'name':'Glacier', 'density'..."
oa,oasis,special,5,0.3,5,an oasis,NaN,raider,"[ {'name':'Palm Tree', 'densit..."


In [5]:
beastiary = bdf.T.to_dict()
beastiary['goblin']

{'attackType': 'swung a rusty sword',
 'color': '#3E8E3E',
 'damage': 4,
 'group max': 10,
 'group min': 5,
 'health': 3,
 'healthMaxVariance': 2,
 'move': 5,
 'move type': 'swarm',
 'name': 'Goblin',
 'perception': 10,
 'render type': 'pack',
 'size': 5}

Adding a sample terrain just to build out the process. mapData in the game comes with the 'area' as well as the 4 neighboring areas (N,S,E,W). In this case I'm only loading 'area' so that I can populate the creatures there. This examply is just statically pasted from the console. 

In [6]:
#just statically loaded, but 
mapData = {'area': {'Code': 'pl',
  'Creature Probability': 0.1,
  'Danger Level': 2,
  'Description': 'a grassy plain',
  'Detailed Description': 'None',
  'Terrain Code': 'pl',
  'Terrain Textures': [{'abundance': 150,
    'affect': 'no',
    'affectAmt': 0,
    'affectText': 'none',
    'density': 5,
    'hex': '#006400',
    'name': 'Grass',
    'size': 1,
    'spread': 'scatter'},
   {'abundance': 5,
    'affect': 'bump',
    'affectAmt': 2,
    'affectText': '*** hit a rock',
    'density': 5,
    'hex': '#2F4F4F',
    'name': 'Rock',
    'size': 5,
    'spread': 'scatter'}],
  'Treasure Level': 2,
  'Type': 'normal',
  'Type Name': 'plain',
  'creatures': 'goblin', #should actually be a tiger, but I changed it for testing.
  'x': 3,
  'y': 3}
    }

In [7]:
mapData['area']

{'Code': 'pl',
 'Creature Probability': 0.1,
 'Danger Level': 2,
 'Description': 'a grassy plain',
 'Detailed Description': 'None',
 'Terrain Code': 'pl',
 'Terrain Textures': [{'abundance': 150,
   'affect': 'no',
   'affectAmt': 0,
   'affectText': 'none',
   'density': 5,
   'hex': '#006400',
   'name': 'Grass',
   'size': 1,
   'spread': 'scatter'},
  {'abundance': 5,
   'affect': 'bump',
   'affectAmt': 2,
   'affectText': '*** hit a rock',
   'density': 5,
   'hex': '#2F4F4F',
   'name': 'Rock',
   'size': 5,
   'spread': 'scatter'}],
 'Treasure Level': 2,
 'Type': 'normal',
 'Type Name': 'plain',
 'creatures': 'goblin',
 'x': 3,
 'y': 3}

## This is the function that will add creatures
Creatures are created each time a character wanders into a new area. The __'monsters'__ key of the mapData contains a dict which contains both metadata *and* the list of monsters, along with all of the info that those monsters will need.  

Here is the process:
* if the 'danger level' is less than 1, there is no danger. 
* if the 'danger level is more than 1, but a random number (0-1) is less than 'creature probability', there is no danger.
* if 'danger level' is more than 1, and the random number is more than 'creature probability', then there are monsters
    * 
    create an array of json objects with the properties of that creature
    

In [8]:
def addMonsters(mapData):
    '''
    takes mapData as DICT
    returns monsters as DICT
    example:
    mapData['monsters'] = addMonsters(mapData) 
    '''
    individualVariables = ["name","move","size","color"]
    monsters = {}
    #beastiary = getBeastiary()   #This line is not used here because I'm not importing the bestiary from S3
    print("Danger level",mapData['area']['Danger Level'])
    if mapData['area']['Danger Level'] < 1:
        #if the danger level is less than 0, don't load any monsters.
        monsters['message'] = 'The area looks calm and peacefull.'
        return monsters
    creature = beastiary[mapData['area']['creatures']]
    metaVariables = [k for k in list(creature.keys()) if k not in individualVariables]
    d10 = np.random.random(1)[0]
    print("random encounter roll = ",d10,". ", mapData['area']['Creature Probability']," needed to appear")
    if d10 > mapData['area']['Creature Probability']:
        #even if there is a danger in the area, that doesn't mean a monster will show.
        monsters['message'] = 'The area looks calm and peacefull.'
        return monsters
    if d10 <= mapData['area']['Creature Probability']:
        monsters['message'] = 'there is a ' + creature['name'] + ' in the area.'
        if creature['group min'] == creature['group max']:
            #if the min and max are the same, the min number will be drawn.
            nMonsters = creature['group min']
        else:
            #otherwise, the number of monsters is a random number between the min and the max.
            nMonsters = np.random.randint(creature['group min'],creature['group max'])
    #metadata about the monsters is useful when they are rendered
    monsters['meta'] = dict([i for i in beastiary[mapData['area']['creatures']].items() if i[0] in metaVariables])
    #a list of monsters is added that is nMonsters long
    m = []
    for i in range(nMonsters):
        mi = dict([i for i in beastiary[mapData['area']['creatures']].items() if i[0] in individualVariables])
        if (('health' in monsters['meta'].keys()) & ('healthMaxVariance' in monsters['meta'].keys())):
            mi['health'] = monsters['meta']['health'] + np.random.randint(0, monsters['meta']['healthMaxVariance'])
        m.append(mi) 
    monsters['m'] = m
    return monsters

In this example, I'm increasing the creature probability to 100% just to show what the file looks like. 

In [9]:
mapData['area']['Creature Probability'] = 1
addMonsters(mapData)

Danger level 2
random encounter roll =  0.43912947779451583 .  1  needed to appear


{'m': [{'color': '#3E8E3E',
   'health': 4,
   'move': 5,
   'name': 'Goblin',
   'size': 5},
  {'color': '#3E8E3E', 'health': 3, 'move': 5, 'name': 'Goblin', 'size': 5},
  {'color': '#3E8E3E', 'health': 3, 'move': 5, 'name': 'Goblin', 'size': 5},
  {'color': '#3E8E3E', 'health': 3, 'move': 5, 'name': 'Goblin', 'size': 5},
  {'color': '#3E8E3E', 'health': 3, 'move': 5, 'name': 'Goblin', 'size': 5},
  {'color': '#3E8E3E', 'health': 4, 'move': 5, 'name': 'Goblin', 'size': 5}],
 'message': 'there is a Goblin in the area.',
 'meta': {'attackType': 'swung a rusty sword',
  'damage': 4,
  'group max': 10,
  'group min': 5,
  'health': 3,
  'healthMaxVariance': 2,
  'move type': 'swarm',
  'perception': 10,
  'render type': 'pack'}}

Note the variance in the health of the goblins.

On the other hand if there is no danger then no creatures are added to the json file:

In [10]:
mapData['area']['Creature Probability'] = 0
addMonsters(mapData)

Danger level 2
random encounter roll =  0.9003413418409705 .  0  needed to appear


{'message': 'The area looks calm and peacefull.'}